In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [Name of the Notebook]

[Change the link]

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_1_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Overview

[add overview]

[add what you learned in previous notebook and link] - skip if its first


[Context of this notebook compared to overall idea]

## Getting Started

### Install Dependencies


In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 47.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Define project information
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_NAME = "mlops-for-genai" # @param {type:"string"}
# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize cloud storage
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

### Import libraries


In [ ]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)
from vertexai.preview.tokenization import get_tokenizer_for_model
import sqlite3
import datetime
import timeit
import datetime
import uuid
import re
import pandas as pd
import os

### Load the models

To learn more about all [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [ ]:
MODEL_ID_PRO = "gemini-1.5-pro-001"  # @param {type:"string"}
MODEL_ID_FLASH = "gemini-1.5-flash-001" # @param {type:"string"}

model_pro = GenerativeModel(MODEL_ID_PRO)
model_flash = GenerativeModel(MODEL_ID_FLASH)

tokenizer_pro = get_tokenizer_for_model(MODEL_ID_PRO)
tokenizer_flash = get_tokenizer_for_model(MODEL_ID_FLASH)

In [ ]:
DB_NAME = "gemini_flow"  # @param {type:"string"}

## Create Local DB with SQLite [Run only once]

In [ ]:
#@title SQLite DB Creation

def setup_and_track_gemini_prompts(db_name="gemini_prompts.db"):
    """
    Sets up a SQLite database to store Vertex Gemini API artifacts and extracts payload details from Gemini calls.

    Args:
        db_name: The name of the SQLite database file (default: "gemini_prompts.db")
    """

    # Create or connect to the database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS gemini_prompts (
            uuid TEXT PRIMARY KEY,  -- Use uuid as the primary key
            timestamp DATETIME,
            prompt_tag TEXT,  -- Store prompt_tag as a comma-separated string
            prompt_note TEXT,
            apx_cost_dollar REAL,
            input_prompt TEXT,
            gcs_path TEXT,
            mime_type TEXT,
            max_output_tokens INTEGER,
            temperature REAL,
            top_p REAL,
            top_k REAL,
            input_prompt_token_count INTEGER,
            model_output TEXT,
            prompt_token_count INTEGER,
            candidates_token_count INTEGER,
            total_token_count INTEGER,
            prompt_feedback TEXT,
            total_processing_time_sec REAL,
            finish_reason TEXT,
            input_safety_settings TEXT,
            output_safety_rating TEXT
        )
    ''')
    print("Table created successfully!")
    conn.commit()
    return conn, cursor

In [ ]:
conn, cursor = setup_and_track_gemini_prompts(db_name=DB_NAME)

Table created successfully!


## Set pricing

In [ ]:
#@title Gemini API Pricing [Update]

pricing = {
    "Gemini 1.5 Flash": {
        "Multimodal": {
            "Image Input": {
                "<= 128K context window": 0.00002,
                "> 128K context window": 0.00004
            },
            "Video Input": {
                "<= 128K context window": 0.00002,
                "> 128K context window": 0.00004
            },
            "Text Input": {
                "<= 128K context window": 0.00001875,
                "> 128K context window": 0.0000375
            },
            "Audio Input": {
                "<= 128K context window": 0.000002,
                "> 128K context window": 0.000004
            },
            "Text Output": {
                "<= 128K context window": 0.000075,
                "> 128K context window": 0.00015
            }
        }
    },
    "Gemini 1.5 Pro": {
        "Multimodal": {
            "Image Input": {
                "<= 128K context window": 0.001315,
                "> 128K context window": 0.00263
            },
            "Video Input": {
                "<= 128K context window": 0.001315,
                "> 128K context window": 0.00263
            },
            "Text Input": {
                "<= 128K context window": 0.00125,
                "> 128K context window": 0.0025
            },
            "Audio Input": {
                "<= 128K context window": 0.000125,
                "> 128K context window": 0.00025
            },
            "Text Output": {
                "<= 128K context window": 0.00375,
                "> 128K context window": 0.0075
            }
        }
    }
}

## Prompt Management




In [ ]:
#@title Prompt Management Helper Functions

def calculate_gemini_cost(usage_metadata, pricing, model="Gemini 1.5 Pro",tokens_per_1k_chars=750):
    """
    Calculates the cost of a Gemini API call based on usage metadata and pricing.

    Args:
        usage_metadata: A dictionary containing 'prompt_token_count', 'candidates_token_count', and 'total_token_count'.
        pricing: A dictionary containing the pricing structure (as extracted from the image).
        model: The Gemini model used (e.g., "Gemini 1.5 Pro", "Gemini 1.5 Flash").

    Returns:
        The total cost of the API call.
    """

    # Determine context window category from total_token_count
    context_window_size = usage_metadata['total_token_count']
    context_window_category = "<= 128K context window" if context_window_size <= 128000 else "> 128K context window"

    # Extract relevant prices
    text_input_price = pricing[model]["Multimodal"]["Text Input"][context_window_category]
    text_output_price = pricing[model]["Multimodal"]["Text Output"][context_window_category]

    # Calculate costs using the provided estimation
    input_cost = (usage_metadata['prompt_token_count'] / tokens_per_1k_chars) * text_input_price
    output_cost = (usage_metadata['candidates_token_count'] / tokens_per_1k_chars) * text_output_price

    total_cost = input_cost + output_cost
    return total_cost

def calculate_gemini_cost(usage_metadata, pricing, model="Gemini 1.5 Pro",tokens_per_1k_chars=750):
    """
    Calculates the cost of a Gemini API call based on usage metadata and pricing.

    Args:
        usage_metadata: A dictionary containing 'prompt_token_count', 'candidates_token_count', and 'total_token_count'.
        pricing: A dictionary containing the pricing structure (as extracted from the image).
        model: The Gemini model used (e.g., "Gemini 1.5 Pro", "Gemini 1.5 Flash").

    Returns:
        The total cost of the API call.
    """

    # Determine context window category from total_token_count

    context_window_size = usage_metadata.total_token_count
    context_window_category = "<= 128K context window" if context_window_size <= 128000 else "> 128K context window"

    # Extract relevant prices
    text_input_price = pricing[model]["Multimodal"]["Text Input"][context_window_category]
    text_output_price = pricing[model]["Multimodal"]["Text Output"][context_window_category]

    # Calculate costs using the provided estimation
    input_cost = (usage_metadata.prompt_token_count / tokens_per_1k_chars) * text_input_price
    output_cost = (usage_metadata.candidates_token_count / tokens_per_1k_chars) * text_output_price

    total_cost = input_cost + output_cost
    return total_cost

def extract_gemini_payload(response, prompt, uri_path, mime_type, generation_config,
                           safety_settings,prompt_token_count,time, prompt_tag, prompt_note, prompt_uuid,
                           query_cost):
  """Extracts relevant payload from a Gemini API response."""
  payload = {
      "uuid": prompt_uuid,
      "timestamp": datetime.datetime.now(),
      "prompt_tag": prompt_tag,
      "prompt_note": prompt_note,
      "apx_cost_dollar": float(query_cost),
      "input_prompt": prompt,  # Assuming you have access to the original prompt
      "gcs_path": uri_path,  # If applicable
      "mime_type": mime_type,  # If applicable
      "max_output_tokens": int(generation_config['max_output_tokens']),
      "temperature": generation_config['temperature'],
      "top_p": float(generation_config['top_p']),
      "top_k": float(generation_config['top_k']),
      "input_prompt_token_count": int(prompt_token_count),
      "model_output": response.text,
      "prompt_token_count": int(response.usage_metadata.prompt_token_count),
      "candidates_token_count": int(response.usage_metadata.candidates_token_count),
      "total_token_count": int(response.usage_metadata.total_token_count),
      "prompt_feedback": str(response.prompt_feedback),
      "total_processing_time_sec": float(time),
      "finish_reason": str(response.candidates[0].finish_reason),
      "input_safety_settings": str(safety_settings),
      "output_safety_rating": str(response.candidates[0].safety_ratings)
  }

  return payload


def payload_upload(payload, db_name="gemini_prompts.db"):
    """
    Uploads the extracted payload to the SQLite database.

    Args:
        payload: The dictionary containing the extracted payload details.
    """

    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Convert prompt_tag list to comma-separated string
        prompt_tag_str = ",".join(payload['prompt_tag']) if payload['prompt_tag'] else None

        # Insert the payload into the database, including new columns and using uuid
        cursor.execute('''
            INSERT INTO gemini_prompts (
                uuid, timestamp, prompt_tag, prompt_note, apx_cost_dollar, input_prompt, gcs_path, mime_type,
                max_output_tokens, temperature, top_p, top_k, input_prompt_token_count,
                model_output, prompt_token_count, candidates_token_count, total_token_count,
                prompt_feedback, total_processing_time_sec, finish_reason,
                input_safety_settings, output_safety_rating

            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?)
        ''', (
            payload['uuid'], payload['timestamp'], prompt_tag_str, payload['prompt_note'], payload['apx_cost_dollar'],
            payload['input_prompt'], payload['gcs_path'], payload['mime_type'],
            payload['max_output_tokens'], payload['temperature'],
            payload['top_p'], payload['top_k'], payload['input_prompt_token_count'],
            payload['model_output'], payload['prompt_token_count'], payload['candidates_token_count'],
            payload['total_token_count'], payload['prompt_feedback'],
            payload['total_processing_time_sec'], payload['finish_reason'], payload['input_safety_settings'],
            payload['output_safety_rating']

        ))

        conn.commit()
        print("Payload uploaded successfully!")

    except sqlite3.Error as e:
        print(f"Error uploading payload: {e}")

    finally:
        conn.close()


def get_gemini_response(model, tokenizer, pricing=None, generation_config=None,
                    safety_settings=None,
                    uri_path=None,mime_type=None, prompt=None,
                    prompt_tag=None, prompt_note=None, database_name=None):

  # Verification for prompt_tag
  if prompt_tag:
    if not isinstance(prompt_tag, list):
      prompt_tag = [prompt_tag]  # Convert single tag to a list
    for tag in prompt_tag:
      if not re.match(r"^\w+(?:_\w+)*$", tag):
        raise ValueError("Invalid prompt_tag format. Use single or double words separated by underscores.")

  # Generate a UUID for the prompt
  prompt_uuid = str(uuid.uuid4())

  token_response = tokenizer.count_tokens(prompt)
  prompt_token_count = token_response.total_tokens
  # print(f"Prompt Token Count: {prompt_token_count}")
  if not generation_config:
    generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
      "top_k": 40,
    }

  if not safety_settings:
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
  start = timeit.default_timer()

  if uri_path and mime_type:
    file = Part.from_uri(
      mime_type=mime_type,
      uri=uri_path
      )
    responses = model.generate_content([file, prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        # stream=True,
    )
  else:
    responses = model.generate_content([prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        # stream=True,
    )
  stop = timeit.default_timer()
  processing_time = stop - start
  cost_of_query = calculate_gemini_cost(responses.usage_metadata, pricing)
  payload = extract_gemini_payload(responses, prompt, uri_path, mime_type, generation_config,
                           safety_settings,prompt_token_count,processing_time, prompt_tag, prompt_note, prompt_uuid,
                                   cost_of_query)

  payload_upload(payload, db_name=database_name)
  print("Payload uploaded to the database: ", database_name)
  return responses.text, payload


def query_gemini_prompts(db_name=None, year=None, month=None, day=None, temperature=None,
                         max_output_tokens=None, top_p=None, top_k=None,
                         total_token_count=None, prompt_token_count=None,
                         total_processing_time_sec=None, finish_reason=None,
                         default_n=None, uuid=None, tags=None,
                         calculate_cost=False):
    """
    Queries the 'gemini_prompts' table based on provided filters and optionally calculates the aggregate cost.

    Args:
        conn: The database connection.
        year, month, day: Filters by year, month, and day of the timestamp.
        temperature, max_output_tokens, top_p, top_k: Filters by generation parameters.
        total_token_count, prompt_token_count: Filters by token counts.
        total_processing_time_sec: Filters by processing time.
        finish_reason: Filters by finish reason.
        default_n: (Optional) The default number of last rows to return. If None, returns all records.
        uuid: (Optional) Filters by a specific UUID.
        tags: (Optional) A list of tags to filter by. Only rows containing ALL of these tags will be returned.
        calculate_cost: (Optional) If True, calculates the aggregate cost based on year, month, and day filters.

    Returns:
        - If calculate_cost is False: A pandas DataFrame representing the query results.
        - If calculate_cost is True: A tuple containing:
            - A pandas DataFrame with the aggregate cost.
            - The total cost.

    Raises:
        ValueError: If calculate_cost is True and none of year, month, or day is provided.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    if calculate_cost and not any([year, month, day]):
        raise ValueError("At least one of year, month, or day must be provided when calculate_cost is True.")

    # Build the SQL query dynamically based on provided filters
    query = "SELECT * FROM gemini_prompts WHERE 1=1"  # Start with a basic query
    params = []
    if uuid:
        query += " AND uuid = ?"
        params.append(uuid)

    if tags:
        for tag in tags:
            query += " AND prompt_tag LIKE ?"  # Use LIKE for partial string matching
            params.append(f"%{tag}%")  # Match tags anywhere within the comma-separated string

    if year:
        query += " AND strftime('%Y', timestamp) = ?"
        params.append(str(year))
    if month:
        query += " AND strftime('%m', timestamp) = ?"
        params.append(str(month).zfill(2))  # Ensure two-digit month format
    if day:
        query += " AND strftime('%d', timestamp) = ?"
        params.append(str(day).zfill(2))  # Ensure two-digit day format

    if temperature:
        query += " AND temperature = ?"
        params.append(temperature)
    if max_output_tokens:
        query += " AND max_output_tokens = ?"
        params.append(max_output_tokens)
    if top_p:
        query += " AND top_p = ?"
        params.append(top_p)
    if top_k:
        query += " AND top_k = ?"
        params.append(top_k)

    if total_token_count:
        query += " AND total_token_count = ?"
        params.append(total_token_count)
    if prompt_token_count:
        query += " AND prompt_token_count = ?"
        params.append(prompt_token_count)

    if total_processing_time_sec:
        query += " AND total_processing_time_sec = ?"
        params.append(total_processing_time_sec)

    if finish_reason:
        query += " AND finish_reason = ?"
        params.append(finish_reason)

    # Add LIMIT clause for default_n
    if default_n:
        query += " ORDER BY timestamp DESC LIMIT ?"  # Order by timestamp descending and limit
        params.append(default_n)

    # Execute the query with the parameters
    cursor.execute(query, params)

    # Fetch the results and column names
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=column_names)

    # Cost Calculation Logic
    if calculate_cost:
        # Assuming you have a 'cost' column in your 'gemini_prompts' table
        cost_query = "SELECT "

        # Dynamically build the GROUP BY clause based on provided filters
        group_by_clause = []
        if year:
            cost_query += "strftime('%Y', timestamp) AS year, "
            group_by_clause.append("year")
        if month:
            cost_query += "strftime('%m', timestamp) AS month, "
            group_by_clause.append("month")
        if day:
            cost_query += "strftime('%d', timestamp) AS day, "
            group_by_clause.append("day")

        cost_query += "SUM(apx_cost_dollar) AS total_cost FROM gemini_prompts WHERE 1=1 "

        # Add the same WHERE clause conditions as the main query
        cost_query += query.split("WHERE 1=1")[1]  # Extract the WHERE clause from the main query

        # Add the GROUP BY clause
        if group_by_clause:
            cost_query += " GROUP BY " + ", ".join(group_by_clause)

        # Execute the cost query
        cursor.execute(cost_query, params)
        cost_results = cursor.fetchall()
        cost_column_names = [desc[0] for desc in cursor.description]
        cost_df = pd.DataFrame(cost_results, columns=cost_column_names)

        # Calculate the total cost
        total_cost = cost_df['total_cost'].sum()

        return total_cost
    else:
        return df



### Gemini API Call with payload

In [ ]:
res, payload = get_gemini_response(model_pro,tokenizer_flash,
                                   pricing=pricing,
                                   prompt="help me make a buisness plan for marketing startup that makes cool shoes",
                                   prompt_tag=['startup','strategy'],
                                   prompt_note="marketing strategy for startups",
                                   database_name = DB_NAME)

Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow


In [ ]:
# payload

In [ ]:
%%time

strategy = ["startup company", "oil company", "ad company", 'manufactoring company', 'technology company']

overall_response = []
for each_cat in strategy:
  res, _ = get_gemini_response(model_pro,tokenizer_flash,
                                    pricing=pricing,
                                    prompt="tell me 5 five best strategy to market a startup",
                                    prompt_tag=['marketing','plan','strategy', "_".join(each_cat.split())],
                                    prompt_note="marketing strategy for each_cat",
                                    database_name = DB_NAME)
  overall_response.append(res)

Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow
Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow
Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow
Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow
Payload uploaded successfully!
Payload uploaded to the database:  gemini_flow
CPU times: user 455 ms, sys: 62.3 ms, total: 518 ms
Wall time: 1min 7s


### DB Query

In [ ]:
# Query by tag

results = query_gemini_prompts(DB_NAME, tags=['oil'])
results

,uuid,timestamp,prompt_tag,prompt_note,apx_cost_dollar,input_prompt,gcs_path,mime_type,max_output_tokens,temperature,...,input_prompt_token_count,model_output,prompt_token_count,candidates_token_count,total_token_count,prompt_feedback,total_processing_time_sec,finish_reason,input_safety_settings,output_safety_rating
0,cda0fc96-59aa-4cf5-bfdb-4cd5549dbeb3,2024-08-11 09:49:20.485395,"marketing,plan,strategy,oil_company",marketing strategy for each_cat,0.003443,tell me 5 five best strategy to market a startup,None,None,8192,1.0,...,11,Here are 5 of the best strategies to market a ...,11,685,696,,13.203109,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...


In [ ]:
# Query by year and month

results = query_gemini_prompts(DB_NAME, year=2024, month=8)
results.head(2)

,uuid,timestamp,prompt_tag,prompt_note,apx_cost_dollar,input_prompt,gcs_path,mime_type,max_output_tokens,temperature,...,input_prompt_token_count,model_output,prompt_token_count,candidates_token_count,total_token_count,prompt_feedback,total_processing_time_sec,finish_reason,input_safety_settings,output_safety_rating
0,6c2d022a-e86e-42a2-bfb6-ce22a7481e73,2024-08-11 09:44:15.742998,"startup,strategy",marketing strategy for startups,0.004835,help me make a buisness plan for marketing sta...,None,None,8192,1.0,...,15,## Business Plan: [Your Startup Name] - Cool S...,15,962,977,,18.574617,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...
1,a227b6cf-291b-4d52-a26b-0a08a751553d,2024-08-11 09:44:51.534141,"startup,strategy",marketing strategy for startups,0.003268,tell me 5 five best strategy to market a startup,None,None,8192,1.0,...,11,Here are 5 of the best strategies to market a ...,11,650,661,,12.692525,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...


In [ ]:
# Query by temperature and max_output_tokens

results = query_gemini_prompts(DB_NAME, temperature=1, max_output_tokens=8192)
results.head(2)

,uuid,timestamp,prompt_tag,prompt_note,apx_cost_dollar,input_prompt,gcs_path,mime_type,max_output_tokens,temperature,...,input_prompt_token_count,model_output,prompt_token_count,candidates_token_count,total_token_count,prompt_feedback,total_processing_time_sec,finish_reason,input_safety_settings,output_safety_rating
0,6c2d022a-e86e-42a2-bfb6-ce22a7481e73,2024-08-11 09:44:15.742998,"startup,strategy",marketing strategy for startups,0.004835,help me make a buisness plan for marketing sta...,None,None,8192,1.0,...,15,## Business Plan: [Your Startup Name] - Cool S...,15,962,977,,18.574617,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...
1,a227b6cf-291b-4d52-a26b-0a08a751553d,2024-08-11 09:44:51.534141,"startup,strategy",marketing strategy for startups,0.003268,tell me 5 five best strategy to market a startup,None,None,8192,1.0,...,11,Here are 5 of the best strategies to market a ...,11,650,661,,12.692525,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...


In [ ]:
# Query all the data

results = query_gemini_prompts(DB_NAME)
print("Total rows: ", len(results))
results.head(2)

Total rows:  11


,uuid,timestamp,prompt_tag,prompt_note,apx_cost_dollar,input_prompt,gcs_path,mime_type,max_output_tokens,temperature,...,input_prompt_token_count,model_output,prompt_token_count,candidates_token_count,total_token_count,prompt_feedback,total_processing_time_sec,finish_reason,input_safety_settings,output_safety_rating
0,6c2d022a-e86e-42a2-bfb6-ce22a7481e73,2024-08-11 09:44:15.742998,"startup,strategy",marketing strategy for startups,0.004835,help me make a buisness plan for marketing sta...,None,None,8192,1.0,...,15,## Business Plan: [Your Startup Name] - Cool S...,15,962,977,,18.574617,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...
1,a227b6cf-291b-4d52-a26b-0a08a751553d,2024-08-11 09:44:51.534141,"startup,strategy",marketing strategy for startups,0.003268,tell me 5 five best strategy to market a startup,None,None,8192,1.0,...,11,Here are 5 of the best strategies to market a ...,11,650,661,,12.692525,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...


In [ ]:
# Query all the data and return n records

results = query_gemini_prompts(DB_NAME, default_n=1)
results

,uuid,timestamp,prompt_tag,prompt_note,apx_cost_dollar,input_prompt,gcs_path,mime_type,max_output_tokens,temperature,...,input_prompt_token_count,model_output,prompt_token_count,candidates_token_count,total_token_count,prompt_feedback,total_processing_time_sec,finish_reason,input_safety_settings,output_safety_rating
0,fab62875-909c-4ba8-941c-3235b7365a73,2024-08-11 09:50:00.562286,"marketing,plan,strategy,technology_company",marketing strategy for each_cat,0.003518,tell me 5 five best strategy to market a startup,None,None,8192,1.0,...,11,Here are 5 of the best strategies to market a ...,11,700,711,,13.690452,FinishReason.STOP,{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: ...,[category: HARM_CATEGORY_HATE_SPEECH\nprobabil...


In [ ]:
# Query all the data and return the approx agg cost of query based on year, month and day

results = query_gemini_prompts(DB_NAME, year=2024,
                               month=8,day=11,
                               calculate_cost=True) #atleast one of year, month & day should be passed

print("Approx cost for this query in $: ", results)

Approx cost for this query in $:  0.04022333333333333


## [Optional] Delete Resources [if not used further]


In [ ]:
# Get a list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('gemini_prompts',)]

In [ ]:
# Drop all tables to clear the data
for table in tables:
    cursor.execute(f"DROP TABLE {table[0]};")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# # Delete the database file
# os.remove(DB_NAME)

## Conclusion

[add conclusions & Next Steps]